In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import censusdata

from scipy import stats

os.environ["CENSUS_API_KEY"] = "d6a829e3229193acd4e98d0f4ad90540cdfe59d5"

In [2]:
zip_codes = [94401,94402,94403,94497,94404,94063,94062,94061,94064,94065,94117,94121,94122,94188,94404,94123,94403,94164,94146,
94497,94108,94112,94134,94158,94127,94126,94103,94159,94134,94107,94102,94110,94118,94132,94128,94112,94104,94116,
94133,94143,94401,94124,94130,94147,94109,94129,94105,94114,94115,94402,94119,94125,94111,94131,94141,94140]

In [3]:
zip_codes_str = [str(i) for i in zip_codes]

In [4]:
def local_census_data(year, zip_code):
    # Specify the state code in the geographic boundary
    zipcode = censusdata.censusgeo([('state', '06'), ('zip code tabulation area', zip_code)])

    # Download the data for the specified variables and geographic boundary
    sfbg_zip = censusdata.download('acs5', year, zipcode,
                                   ['B01003_001E','B06009_005E','B06009_006E','B25001_001E','B25002_002E','B25002_003E',
                                    'B19013_001E'],key='d6a829e3229193acd4e98d0f4ad90540cdfe59d5')
    sfbg_zip = sfbg_zip.reset_index()
    sfbg_zip = sfbg_zip.rename(columns={
        'index': 'zip',
        'B01003_001E': 'Total Population',
        'B06009_005E': 'Bachelor degree',
        'B06009_006E': 'Graduate or Professional Degree',
        'B25001_001E': 'Total Housing',
        'B25002_002E': 'Owner-occupied housing units',
        'B25002_003E': 'Renter-occupied housing units',
        'B19013_001E': 'Median Household Income (USD)'
    })
    sfbg_zip['Percentage of Rent'] = sfbg_zip['Renter-occupied housing units']/sfbg_zip['Total Housing']
    sfbg_zip['Bachelor or higher'] = sfbg_zip['Bachelor degree']+sfbg_zip['Graduate or Professional Degree']
    sfbg_zip['Zip Code'] = sfbg_zip['Zip Code'].astype(str)
    sfbg_zip['Zip Code'] = sfbg_zip['Zip Code'].apply(lambda x: str(x).split(">")[-1].split(":")[1].strip() 
                                                                if isinstance(x, str) else x.tract.split(":")[-1].strip())
    sfbg_zip['Year'] = year
    sfbg_zip = sfbg_zip.sort_values(by = ['Zip Code'], ignore_index = True)
    return sfbg_zip


In [6]:
# Concatenate
sf_all = pd.DataFrame()
for year in range(2011, 2020):
    for zip_c in zip_codes_str:
        try:
            sf_county = local_census_data(year, zip_c)
            sf_all = pd.concat([sf_all, sf_county], ignore_index=True)
        except Exception:
            pass

In [7]:
sf_all

,zip,Total Population,Bachelor degree,Graduate or Professional Degree,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Median Household Income (USD),Percentage of Rent,Bachelor or higher,Year
0,94401,34845,5319,2492,13606,12693,913,72452,0.067103,7811,2011
1,94402,23408,5405,4267,9617,9356,261,109093,0.027139,9672,2011
2,94403,39102,8135,4639,16190,15266,924,89205,0.057072,12774,2011
3,94404,32986,8415,6816,13882,13157,725,112271,0.052226,15231,2011
4,94063,30788,1700,989,9340,8877,463,52220,0.049572,2689,2011
...,...,...,...,...,...,...,...,...,...,...,...
373,94114,34918,12186,10977,18384,16845,1539,162193,0.083714,23163,2019
374,94115,34604,11620,8413,19323,17062,2261,123037,0.117011,20033,2019
375,94402,25703,6114,5840,10104,9542,562,169086,0.055622,11954,2019
376,94111,3611,1038,1264,2743,2211,532,120905,0.193948,2302,2019


**Fact Check for 2010-2019 SF MSA, by zip-code**

In [32]:
grouped_data.groupby('Year').agg({
    'Total Population': 'sum',
    'Bachelor or higher': 'sum',
    'Total Housing': 'sum',
    'Owner-occupied housing units': 'sum',
    'Renter-occupied housing units': 'sum',
    'Median Household Income (USD)': 'median'
}).reset_index()

,Year,Total Population,Bachelor or higher,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Median Household Income (USD)
0,2011,1281428,457558,554873,509007,45866,75965.0
1,2012,1297970,468409,555583,511831,43752,76461.0
2,2013,1314266,480011,560988,519051,41937,81939.5
3,2014,1333094,496119,564354,524041,40313,84015.0
4,2015,1351708,515072,568665,529994,38671,87321.0
5,2016,1366787,535111,573260,535392,37868,95643.5
6,2017,1388976,559119,579233,539348,39885,101883.5
7,2018,1397045,579725,584169,539857,44312,109911.5
8,2019,1403200,593849,588763,543535,45228,121274.5


In [12]:
sf_all = sf_all.rename(columns = {'zip':'Zip Code'})

In [30]:
grouped_data = sf_all.groupby(['Zip Code','Year']).agg({
    'Total Population': 'sum',
    'Bachelor or higher': 'sum',
    'Total Housing': 'sum',
    'Owner-occupied housing units': 'sum',
    'Renter-occupied housing units': 'sum',
    'Median Household Income (USD)': 'median'
}).reset_index()

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [31]:
grouped_data

,Zip Code,Year,Total Population,Bachelor or higher,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Median Household Income (USD)
0,94061,2011,35697,9631,14055,13497,558,74097.0
1,94061,2012,36343,9738,13953,13521,432,75033.0
2,94061,2013,37176,9901,13872,13510,362,79833.0
3,94061,2014,37667,9918,14004,13576,428,81907.0
4,94061,2015,37809,10345,14244,13875,369,85164.0
...,...,...,...,...,...,...,...,...
319,94404,2015,70984,34770,29224,28046,1178,121875.0
320,94404,2016,72172,37042,30372,29128,1244,126618.0
321,94404,2017,73146,37492,30364,28896,1468,134056.0
322,94404,2018,73810,37816,30298,28520,1778,143109.0


In [34]:
grouped_data.to_csv('Y_SF_ACS_2010-2019.csv', index=False)

note: 
    
no 2008 data, 

2009 176 rows; 

2010-2019 197 rows;

2020 forward, some missing data, and 224 rows. 

advice: use 2010-2019 most appropriate

note: I haven't deal with missing values/abnoramlity

# Perform Linear Interpolation

In [58]:
## Create DF

df = (pd
        .DataFrame( index= pd.date_range( '2010-01-01', '2019-12-31' ,freq='MS'))
      .reset_index()
      .rename(columns={'index' : 'y_dt'})
      .assign(key=1)
     )

## load data from Jennifer
df_to_merge = grouped_data

df_census_tract = df_to_merge.loc[:, ['Zip Code']].drop_duplicates().assign(key=1)

df = pd.merge( df, df_census_tract, how='outer', on='key').drop(['key'], axis=1)

assert df_to_merge.shape[0] == df_to_merge.loc[:, ['Zip Code', 'Year']].drop_duplicates().shape[0]

## Deal with assert
df_to_merge.loc[:, 'y_dt'] = pd.to_datetime(df_to_merge['Year'], format='%Y')
df_to_merge = df_to_merge.sort_values('y_dt')

/var/folders/kz/dhtdp71n1q98lc7x715t7c0h0000gp/T/ipykernel_9500/2953862047.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_to_merge.loc[:, 'y_dt'] = pd.to_datetime(df_to_merge['Year'], format='%Y')


In [59]:
df_to_merge.head()
merged_df = pd.merge( df, df_to_merge, how='left', on = ['y_dt', 'Zip Code'])

In [60]:
merged_df = merged_df.sort_values(['Zip Code', 'y_dt'])

new_df = merged_df.copy()
new_df.loc[:, 'new_col'] = np.nan

for census_tract in merged_df.loc[:, 'Zip Code'].unique():

    merged_df = merged_df.sort_values(['Zip Code', 'y_dt'])

    for col in ['Total Population', 'Bachelor or higher', 'Total Housing', 'Owner-occupied housing units',
                'Renter-occupied housing units', 'Median Household Income (USD)']:

        temp_df = (merged_df.loc[(merged_df.loc[:, 'Zip Code'] == census_tract), [col]]
                    .interpolate(method='linear')
                    .interpolate(method='bfill')
                    .reset_index(drop=True))
        
        new_df.loc[(new_df.loc[:, 'Zip Code'] == census_tract), f'Monthly {col}'] = temp_df.values


In [61]:
new_df

,y_dt,Zip Code,Year,Total Population,Bachelor or higher,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Median Household Income (USD),new_col,Monthly Total Population,Monthly Bachelor or higher,Monthly Total Housing,Monthly Owner-occupied housing units,Monthly Renter-occupied housing units,Monthly Median Household Income (USD)
0,2010-01-01,94061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35697.0,9631.0,14055.0,13497.0,558.0,74097.0
36,2010-02-01,94061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35697.0,9631.0,14055.0,13497.0,558.0,74097.0
72,2010-03-01,94061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35697.0,9631.0,14055.0,13497.0,558.0,74097.0
108,2010-04-01,94061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35697.0,9631.0,14055.0,13497.0,558.0,74097.0
144,2010-05-01,94061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35697.0,9631.0,14055.0,13497.0,558.0,74097.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2019-08-01,94404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73970.0,38326.0,30512.0,28682.0,1830.0,149836.0
4211,2019-09-01,94404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73970.0,38326.0,30512.0,28682.0,1830.0,149836.0
4247,2019-10-01,94404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73970.0,38326.0,30512.0,28682.0,1830.0,149836.0
4283,2019-11-01,94404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73970.0,38326.0,30512.0,28682.0,1830.0,149836.0


In [62]:
new_df = new_df.drop(columns=['new_col','Year', 'Total Population','Bachelor or higher','Total Housing',
                     'Owner-occupied housing units',
                'Renter-occupied housing units','Median Household Income (USD)'])
new_df.iloc[:, 2:8] = new_df.iloc[:, 2:8].astype(int)
new_df['Monthly Percentage of Rent'] = new_df['Monthly Renter-occupied housing units'] / new_df['Monthly Total Housing']

In [68]:
new_df['Bachelor or higher Precentage'] = new_df['Monthly Bachelor or higher'] / new_df['Monthly Total Population']

In [70]:
new_df

,y_dt,Zip Code,Monthly Total Population,Monthly Bachelor or higher,Monthly Total Housing,Monthly Owner-occupied housing units,Monthly Renter-occupied housing units,Monthly Median Household Income (USD),Monthly Percentage of Rent,Bachelor or higher Precentage
0,2010-01-01,94061,35697,9631,14055,13497,558,74097,0.039701,0.269799
36,2010-02-01,94061,35697,9631,14055,13497,558,74097,0.039701,0.269799
72,2010-03-01,94061,35697,9631,14055,13497,558,74097,0.039701,0.269799
108,2010-04-01,94061,35697,9631,14055,13497,558,74097,0.039701,0.269799
144,2010-05-01,94061,35697,9631,14055,13497,558,74097,0.039701,0.269799
...,...,...,...,...,...,...,...,...,...,...
4175,2019-08-01,94404,73970,38326,30512,28682,1830,149836,0.059976,0.518129
4211,2019-09-01,94404,73970,38326,30512,28682,1830,149836,0.059976,0.518129
4247,2019-10-01,94404,73970,38326,30512,28682,1830,149836,0.059976,0.518129
4283,2019-11-01,94404,73970,38326,30512,28682,1830,149836,0.059976,0.518129


In [71]:
new_df.to_csv('M_SF_ACS_2010-2019.csv', index=False)